In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from contextlib import contextmanager 
from io import BytesIO 
from tempfile import NamedTemporaryFile 
from transformers import PretrainedConfig, PreTrainedModel 
from transformers import set_seed
from transformers import AdamW, BertForSequenceClassification, BertTokenizer, AutoTokenizer
from transformers import TextClassificationPipeline
from tqdm import tqdm
import json
import torch
import boto3
from psycopg2.extras import RealDictCursor
import psycopg2
import boto3
from psycopg2 import sql
from datetime import datetime
import json
import utils
import pandas as pd
import pathlib

curr = ['43-2', '43-1']

ModuleNotFoundError: No module named 'openaq'

Load data from XML files

In [2]:
db_host = 'openparl.cia2zobysfwo.us-west-2.rds.amazonaws.com'
db_port = 5432
db_user = 'rds'
db_region = 'us-west-2'
db_name = 'openparl'

client = boto3.client('rds', db_region)

db_token = client.generate_db_auth_token(db_host, db_port, db_user, Region=db_region)

connection = psycopg2.connect(database=db_name, host=db_host, user=db_user, password=db_token)

cursor = connection.cursor(cursor_factory=RealDictCursor)

Load Machine Learning Model 

In [3]:
@contextmanager 
def s3_fileobj(bucket, key): 
    """
    Yields a file object from the filename at {bucket}/{key}

    Args:
        bucket (str): Name of the S3 bucket where you model is stored
        key (str): Relative path from the base of your bucket, including the filename and extension of the object to be retrieved.
    """
    s3 = boto3.client("s3") 
    obj = s3.get_object(Bucket=bucket, Key=key) 
    yield BytesIO(obj["Body"].read()) 
 
def load_model(bucket, path_to_model, model_name='pytorch_model'):
    """
    Load a model at the given S3 path. It is assumed that your model is stored at the key:

        '{path_to_model}/{model_name}.bin'

    and that a config has also been generated at the same path named:

        f'{path_to_model}/config.json'

    IF YOU'RE ON WINDOWS OS: make sure the delete parameter in NamedTemporaryFile is set to False

    """
    tempfile = NamedTemporaryFile(delete=False) 
    with s3_fileobj(bucket, f'{path_to_model}/{model_name}.bin') as f: 
        tempfile.write(f.read())
    print('Done Writing')
 
    with s3_fileobj(bucket, f'{path_to_model}/config.json') as f: 
        dict_data = json.load(f) 
        print('Done loading Data Dict')
        config = PretrainedConfig.from_dict(dict_data) 
        print('Done config file')
 
    model = BertForSequenceClassification.from_pretrained(tempfile.name, config=config) 
    return model 
    
def categorize(text_data):
    try:
        if len(text_data) > 1800:
            text_data = text_data[:1600]
        label = pipe(text_data)
        if label[0]['score'] < threshold:
            label = 'informal'
        else:
            label = label[0]['label']
            label = lab2ind[label]
        return label
    except RuntimeError:
        return 'informal'

lab2ind = {
            'LABEL_0': 'economy',
            'LABEL_1': 'environment',
            'LABEL_2': 'foreign policy',
            'LABEL_3': 'government',
            'LABEL_4': 'infrastructure',
            'LABEL_5': 'law',
            'LABEL_6': 'welfare'
        }

threshold = 0.2 # minimum required percentage for model to categorize speech, else labeled informal

model = load_model('bert-topic-classifier-june-2022', 'transcript_processing')

set_seed(42)

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=False)


Done Writing
Done loading Data Dict
Done config file


In [5]:
path =("./Text//Seattle.csv")
data = pd.read_csv(path,sep=' ',header=None)
for i, row in data.iterrows():
    if str(row[1]).startswith('Minutes of'):
        data.drop(i,axis=0,inplace=True)
len(data)

1109

In [6]:
for i, row in data.iterrows():
    data.at[i , 'topic'] = categorize(str(row[1]))
# pd.DataFrame(data)
print('done!')
print(data)

Token indices sequence length is longer than the specified maximum sequence length for this model (963 > 512). Running this sequence through the model will result in indexing errors


done!
                 0                                                  1  \
0     Jul 12, 2022  [<p class="navbar-text navbar-right" id="mainM...   
1     Jul  5, 2022  [<p class="navbar-text navbar-right" id="mainM...   
2     Jun 28, 2022  [<p class="navbar-text navbar-right" id="mainM...   
3     Jun 21, 2022  [<p class="navbar-text navbar-right" id="mainM...   
4     Jun 14, 2022  [<p class="navbar-text navbar-right" id="mainM...   
...            ...                                                ...   
1104  Jan 29, 2001  [<p class="navbar-text navbar-right" id="mainM...   
1105  Jan 22, 2001  [<p class="navbar-text navbar-right" id="mainM...   
1106  Jan 16, 2001  [<p class="navbar-text navbar-right" id="mainM...   
1107  Jan  8, 2001  [<p class="navbar-text navbar-right" id="mainM...   
1108  Jan  2, 2001  [<p class="navbar-text navbar-right" id="mainM...   

         topic  
0     informal  
1     informal  
2     informal  
3     informal  
4     informal  
...        ... 

In [118]:
data.to_csv(path,header=None, index=None, sep=' ')

SO output data for speaker stat blocks has been made, time to upload data :)

In [119]:
db_host = 'openparl.cia2zobysfwo.us-west-2.rds.amazonaws.com'
db_port = 5432
db_user = 'rds'
db_region = 'us-west-2'
db_name = 'openparl'

client = boto3.client('rds', db_region)

db_token = client.generate_db_auth_token(db_host, db_port, db_user, Region=db_region)

connection = psycopg2.connect(database=db_name, host=db_host, user=db_user, password=db_token)

cursor = connection.cursor(cursor_factory=RealDictCursor)

In [120]:
table = 'us_sea_gen_meeting'
data.columns = ['meeting_date', 'meeting_minutes','topic']
df_dict = data.to_dict('records')
try:
    make_table = sql.SQL(""" 
        CREATE TABLE IF NOT EXISTS {table} (
            meeting_date text,
            meeting_minutes text,
            topic text
        );

        ALTER TABLE {table} OWNER TO rds_ad;
    """).format(table = sql.Identifier(table))

    cursor.execute(make_table)
    cursor.connection.commit() 
except Exception as e:
    print(f"An exception occured while making a table:\n{e}")
    cursor.connection.rollback()

insert_query = sql.SQL(""" 
    INSERT INTO {table}
    VALUES (%s, %s, %s)
    ON CONFLICT DO NOTHING
""").format(table=sql.Identifier(table))

counter = 0
for item in df_dict:
    try:
            tup = (
                item['meeting_date'],
                item['meeting_minutes'],
                item['topic']
            )
            cursor.execute(insert_query, tup)
            cursor.connection.commit()
            counter += 1
            if counter % 100 == 0:
                done = round(counter / len(data), 4) * 100
                print(f"Done {done} %")
    except Exception as e:
        print(f"An exception occured while inserting data: \n{e}")
        cursor.connection.rollback()
print('Done!')

Done 1.9800000000000002 %
Done 3.9600000000000004 %
Done 5.949999999999999 %
Done 7.93 %
Done 9.91 %
Done 11.89 %
Done 13.88 %
Done 15.86 %
Done 17.84 %
Done 19.82 %
Done 21.8 %
Done 23.79 %
Done 25.77 %
Done 27.750000000000004 %
Done 29.73 %
Done 31.71 %
Done 33.7 %
Done 35.68 %
Done 37.66 %
Done 39.64 %
Done 41.63 %
Done 43.61 %
Done 45.59 %
Done 47.57 %
Done 49.55 %
Done 51.54 %
Done 53.52 %
Done 55.50000000000001 %
Done 57.48 %
Done 59.46 %
Done 61.45 %
Done 63.43 %
Done 65.41 %
Done 67.39 %
Done 69.38 %
Done 71.36 %
Done 73.34 %
Done 75.32 %
Done 77.3 %
Done 79.29 %
Done 81.27 %
Done 83.25 %
Done 85.22999999999999 %
Done 87.22 %
Done 89.2 %
Done 91.18 %
Done 93.16 %
Done 95.14 %
Done 97.13000000000001 %
Done 99.11 %
Done!
